In [1]:
import math
import numpy as np
from CGC import *
import pandas as pd
import time

In [2]:
def dfact(n):
    if n>1:
        return fact(n)/fact((n-1)/2)/math.pow(2,(n-1)/2)
    else:
        return 1
    
def gauss(power, coef):
    if power % 2 == 0:
        return (dfact(power-1)* math.sqrt(math.pi)/
                math.pow(2,0.5*power+1) / math.pow(coef, 0.5*power+0.5))
    else:
        return 0.5*fact(0.5*power-0.5)/ math.pow(coef, 0.5*power+0.5)

In [3]:
wf = pd.read_csv("be9.dat", sep="\t", names = ['config','lam', 'lmal', 'LL','SS', 'JJ', 'alpha', 'beta', 'cgamma'])

In [4]:
wf

,config,lam,lmal,LL,SS,JJ,alpha,beta,cgamma
0,0,0,1,1,0.5,1.5,0.045516,0.014496,1.192000e-04
1,0,0,1,1,0.5,1.5,0.045516,0.045018,1.819700e-03
2,0,0,1,1,0.5,1.5,0.045516,0.080839,1.014500e-03
3,0,0,1,1,0.5,1.5,0.045516,0.128650,3.827900e-03
4,0,0,1,1,0.5,1.5,0.045516,0.204750,-4.349200e-03
...,...,...,...,...,...,...,...,...,...
229,6,4,3,2,0.5,1.5,0.604580,0.646790,-5.027300e-04
230,6,4,3,2,0.5,1.5,2.030900,0.025591,4.339800e-08
231,6,4,3,2,0.5,1.5,2.030900,0.085964,-1.769500e-06
232,6,4,3,2,0.5,1.5,2.030900,0.192550,8.356300e-05


In [5]:
def radius():
    
    s = x1 = y1 = 0
    for i in range(len(wf.lam)):
        for j in range(len(wf.lam)):
            if wf.config[i]==wf.config[j]:
                s += (wf.cgamma[i]*wf.cgamma[j]*gauss(wf.lam[i]+wf.lam[j]+2, wf.alpha[i]+wf.alpha[j])
                  *gauss(wf.lmal[i]+wf.lmal[j]+2, wf.beta[i]+wf.beta[j])) 
                x1 += (wf.cgamma[i]*wf.cgamma[j]*gauss(wf.lam[i]+wf.lam[j]+4, wf.alpha[i]+wf.alpha[j])*
                 gauss(wf.lmal[i]+wf.lmal[j]+2, wf.beta[i]+wf.beta[j])) 
                y1 += (wf.cgamma[i]*wf.cgamma[j]*gauss(wf.lam[i]+wf.lam[j]+2, wf.alpha[i]+wf.alpha[j])*
                 gauss(wf.lmal[i]+wf.lmal[j]+4, wf.beta[i]+wf.beta[j])) 
    print('Normirovka = ', s)
    print('R_ch =', round(math.sqrt(1.71**2 + x1/4 + y1/81)*100)/100, 'Fm')

In [6]:
def quad():
    quadrupole = 0; Za = 2;
    e1 = [0, 1, 2]
    e2 = [2, 1, 0]
    for i in range(len(wf.lam)):
        for j in range(len(wf.lam)):
            for k in range(len(e1)):
                quadrupole += (320*math.sqrt(6)*Za*math.pi
    *CG(wf.JJ[i],wf.JJ[i],2,0,wf.JJ[j],wf.JJ[j])*math.sqrt(2*wf.JJ[i]+1)
    *wf.cgamma[i]*wf.cgamma[j]*math.pow(0.5,e1[k])*math.pow(1/9,e2[k])
    *((1+math.pow(-1,e1[k]))/2/math.sqrt(fact(2*e1[k]+1)*fact(2*e2[k]+1)))
    *gauss(wf.lam[i]+wf.lam[j]+e1[k]+2, wf.alpha[i]+wf.alpha[j])
    *gauss(wf.lmal[i]+wf.lmal[j]+e2[k]+2, wf.beta[i]+wf.beta[j])
    *math.pow(-1, wf.JJ[i]+wf.LL[j]+wf.SS[i])
    *math.sqrt(5*(2*wf.LL[i]+1)*(2*wf.LL[j]+1))
    *ninej(wf.lam[i],wf.lmal[i],wf.LL[i],e1[k],e2[k],2,wf.lam[j],wf.lmal[j],wf.LL[j])
    *sixj(wf.SS[i],wf.LL[i],wf.JJ[i],2,wf.JJ[j],wf.LL[j])
    *math.sqrt((2*e1[k]+1)*(2*wf.lam[i]+1)/4/math.pi)*CG(wf.lam[i],0,e1[k],0,wf.lam[j],0)
    *math.sqrt((2*e2[k]+1)*(2*wf.lmal[i]+1)/4/math.pi)*CG(wf.lmal[i],0,e2[k],0,wf.lmal[j],0))
    print('Q =',round(quadrupole*100)/100, 'mbn')

In [7]:
start = time.time()
radius()
quad()
end = time.time() - start
print('time = ', round(end*10000)/10000, 'sec')

Normirovka =  0.9999795850911732
R_ch = 2.35 Fm
Q = 37.44 mbn
time =  44.7076 sec
